In [1]:
import time
import threading
import logging
#import retro
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import initializers
from sklearn.preprocessing import scale
import numpy as np
import cv2
import numpy as np
#import keyboard
import gym
import matplotlib.pylab as plt
from IPython.display import clear_output
#retro.data.list_games()

In [44]:
class VAE():
  def __init__(self, action_space):
    self.action_space = len(action_space)
    image_input = layers.Input(shape=(84,84, 1))
    #preprocessor = layers.experimental.preprocessing.Resizing(84, 84, interpolation='bilinear', name=None)(image_input)
    # Convolutions on the frames on the screen
    #data_format='channels_first'
    layer1 = layers.Conv2D(32, 8, strides=4, activation="relu")(image_input)
    layer2 = layers.Conv2D(64, 4, strides=2, activation="relu")(layer1)
    layer3 = layers.Conv2D(64, 3, strides=1, activation="relu")(layer2)
    layer4 = layers.Flatten()(layer3)
    layer6 = layers.Dense(512, activation="relu")(layer4)
    layer7 = layers.Dense(3, activation="relu")(layer6)
    upsample = layers.Dense(7*7*64)(layer7)
    layer8 = layers.Reshape((7, 7, 64))(upsample)
    layer9 = layers.Conv2DTranspose(64,3, activation="relu")(layer8)
    layer10 = layers.Conv2DTranspose(32,12, activation="relu")(layer9)
    layer11 = layers.Conv2DTranspose(1,65, activation="relu")(layer10)

    #Define NN parameters.
    self.toymodel = keras.Model(inputs=image_input, outputs=layer11)
    self.loss_fn = tf.keras.losses.Huber()
    self.optimizer = keras.optimizers.Adam(learning_rate=0.0000625)
    self.toymodel.compile(self.optimizer, self.loss_fn)

  def trainStep(self, sample_X, sample_Y):
    with tf.GradientTape() as tape:
      old_q = self.toymodel(sample_X, training=True)
      loss_value = self.loss_fn(sample_Y, old_q)
    grads = tape.gradient(loss_value, self.toymodel.trainable_weights)
    self.optimizer.apply_gradients(zip(grads, self.toymodel.trainable_weights))
    return loss_value.numpy()

  def train(self, x_input, y_input, batchsize=64):
    loss_history = []
    dataset = tf.data.Dataset.from_tensor_slices((x_input, y_input))
    dataset = dataset.shuffle(buffer_size=1024).batch(batchsize)
    for steps, (x, y) in enumerate(dataset):
      loss_history.append(self.trainStep(x,y))
    return loss_history

  def forward(self, x_input):
    return self.toymodel(x_input)

In [45]:
env = gym.make('BreakoutNoFrameskip-v4')
action_space = [0,2,3]
test = VAE(action_space)
test.toymodel.summary()

Model: "functional_15"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_31 (InputLayer)        [(None, 84, 84, 1)]       0         
_________________________________________________________________
conv2d_41 (Conv2D)           (None, 20, 20, 32)        2080      
_________________________________________________________________
conv2d_42 (Conv2D)           (None, 9, 9, 64)          32832     
_________________________________________________________________
conv2d_43 (Conv2D)           (None, 7, 7, 64)          36928     
_________________________________________________________________
flatten_11 (Flatten)         (None, 3136)              0         
_________________________________________________________________
dense_24 (Dense)             (None, 512)               1606144   
_________________________________________________________________
dense_25 (Dense)             (None, 3)               

In [ ]:
behavior = keras.models.load_model('D:\\ML_projects\\VGBOT\\210414_1_An_DQN_target')

In [ ]:
evaluate_NN(env, 0.01, 10)